In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import os
import tensorflow_data_validation as tfdv
import tensorflow_model_analysis as tfma
import tempfile
from tfx.components.evaluator.component import Evaluator
from tfx.components.example_gen.csv_example_gen.component import CsvExampleGen
from tfx.components.example_validator.component import ExampleValidator
from tfx.components.model_validator.component import ModelValidator
from tfx.components.pusher.component import Pusher
from tfx.components.schema_gen.component import SchemaGen
from tfx.components.statistics_gen.component import StatisticsGen
from tfx.components.trainer.component import Trainer
from tfx.components.transform.component import Transform
from tfx.orchestration.interactive.interactive_context import InteractiveContext
from tfx.proto import example_gen_pb2
from tfx.proto import evaluator_pb2
from tfx.proto import pusher_pb2
from tfx.proto import trainer_pb2
from tfx.utils.dsl_utils import external_input
from tfx.utils.types import get_split_uri

W0903 20:05:33.852458 139893079226176 deprecation_wrapper.py:119] From /opt/interactive_pipeline/venv/lib/python3.6/site-packages/tfx/components/transform/executor.py:57: The name tf.FixedLenFeature is deprecated. Please use tf.io.FixedLenFeature instead.

W0903 20:05:33.854870 139893079226176 deprecation.py:323] From /opt/interactive_pipeline/venv/lib/python3.6/site-packages/tfx/components/transform/executor.py:57: from_feature_spec (from tensorflow_transform.tf_metadata.dataset_schema) is deprecated and will be removed in a future version.
Instructions for updating:
from_feature_spec is a deprecated, use schema_utils.schema_from_feature_spec


In [3]:
_root = '.'
_data_root = os.path.join(_root, 'data')
_serving_model_dir = os.path.join(tempfile.mkdtemp(),'serving_model/iris')

In [4]:
context = InteractiveContext()

In [5]:
input_split = example_gen_pb2.Input(splits=[
    example_gen_pb2.Input.Split(name='train', pattern='iris_training.csv'),
    example_gen_pb2.Input.Split(name='eval', pattern='iris_test.csv')
])
examples = external_input(_data_root)
example_gen = CsvExampleGen(input_base=examples, input_config=input_split)

context.run(example_gen)

W0903 20:05:34.016581 139893079226176 deprecation_wrapper.py:119] From /opt/interactive_pipeline/venv/lib/python3.6/site-packages/tfx/orchestration/component_launcher.py:87: The name tf.logging.info is deprecated. Please use tf.compat.v1.logging.info instead.

W0903 20:05:34.084782 139893079226176 deprecation_wrapper.py:119] From /opt/interactive_pipeline/venv/lib/python3.6/site-packages/tfx/components/base/base_driver.py:44: The name tf.gfile.Exists is deprecated. Please use tf.io.gfile.exists instead.

W0903 20:05:34.122194 139893079226176 deprecation_wrapper.py:119] From /opt/interactive_pipeline/venv/lib/python3.6/site-packages/tfx/components/example_gen/csv_example_gen/executor.py:83: The name tf.gfile.Glob is deprecated. Please use tf.io.gfile.glob instead.



ExecutionResult(
    component_id: CsvExampleGen
    execution_id: 1
    outputs:
        examples: Channel(
            type_name: ExamplesPath
            artifacts: [Artifact(type_name: ExamplesPath, uri: /tmp/tfx-interactive-2019-09-03T20_05_34.006649-8pq9i6jt/CsvExampleGen/examples/1/train/, split: train, id: 2)
            Artifact(type_name: ExamplesPath, uri: /tmp/tfx-interactive-2019-09-03T20_05_34.006649-8pq9i6jt/CsvExampleGen/examples/1/eval/, split: eval, id: 3)]
        ))

In [6]:
statistics_gen = StatisticsGen(input_data=example_gen.outputs['examples'])
 
context.run(statistics_gen)

ExecutionResult(
    component_id: StatisticsGen
    execution_id: 2
    outputs:
        output: Channel(
            type_name: ExampleStatisticsPath
            artifacts: [Artifact(type_name: ExampleStatisticsPath, uri: /tmp/tfx-interactive-2019-09-03T20_05_34.006649-8pq9i6jt/StatisticsGen/output/2/train/, split: train, id: 4)
            Artifact(type_name: ExampleStatisticsPath, uri: /tmp/tfx-interactive-2019-09-03T20_05_34.006649-8pq9i6jt/StatisticsGen/output/2/eval/, split: eval, id: 5)]
        ))

In [7]:
artifact_list = statistics_gen.outputs['output'].get()
train_artifact_uri = get_split_uri(artifact_list, 'train')
train_stats_path = os.path.join(train_artifact_uri, 'stats_tfrecord')
eval_artifact_uri = get_split_uri(artifact_list, 'eval')
eval_stats_path = os.path.join(eval_artifact_uri, 'stats_tfrecord')

W0903 20:05:44.066306 139893079226176 deprecation.py:323] From <ipython-input-7-b7b1d7d201be>:2: get_split_uri (from tfx.utils.types) is deprecated and will be removed in a future version.
Instructions for updating:
tfx.utils.types.get_split_uri has been renamed to tfx.types.artifact_utils.get_split_uri as of TFX 0.14.0.


In [8]:
train_stats = tfdv.load_statistics(train_stats_path)
eval_stats = tfdv.load_statistics(eval_stats_path)

tfdv.visualize_statistics(lhs_statistics=eval_stats, rhs_statistics=train_stats,
                          lhs_name='EVAL_DATASET', rhs_name='TRAIN_DATASET')

W0903 20:05:44.076256 139893079226176 deprecation.py:323] From /opt/interactive_pipeline/venv/lib/python3.6/site-packages/tensorflow_data_validation/utils/stats_gen_lib.py:357: tf_record_iterator (from tensorflow.python.lib.io.tf_record) is deprecated and will be removed in a future version.
Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


In [9]:
infer_schema = SchemaGen(stats=statistics_gen.outputs['output'])

context.run(infer_schema)

W0903 20:05:44.144880 139893079226176 deprecation_wrapper.py:119] From /opt/interactive_pipeline/venv/lib/python3.6/site-packages/tfx/utils/io_utils.py:76: The name tf.gfile.ListDirectory is deprecated. Please use tf.io.gfile.listdir instead.



ExecutionResult(
    component_id: SchemaGen
    execution_id: 3
    outputs:
        output: Channel(
            type_name: SchemaPath
            artifacts: [Artifact(type_name: SchemaPath, uri: /tmp/tfx-interactive-2019-09-03T20_05_34.006649-8pq9i6jt/SchemaGen/output/3/, split: , id: 6)]
        ))

In [10]:
schema_dir = infer_schema.outputs['output'].get()[0].uri
schema_path = os.path.join(schema_dir, 'schema.pbtxt')

schema = tfdv.load_schema_text(schema_path)
tfdv.display_schema(schema)

,Type,Presence,Valency,Domain
Feature name,,,,
'sepal_width',FLOAT,required,single,-
'petal_width',FLOAT,required,single,-
'species',INT,required,single,-
'sepal_length',FLOAT,required,single,-
'petal_length',FLOAT,required,single,-


In [11]:
validate_stats = ExampleValidator(stats=statistics_gen.outputs['output'],
                                  schema=infer_schema.outputs['output'])

context.run(validate_stats)

ExecutionResult(
    component_id: ExampleValidator
    execution_id: 4
    outputs:
        output: Channel(
            type_name: ExampleValidationPath
            artifacts: [Artifact(type_name: ExampleValidationPath, uri: /tmp/tfx-interactive-2019-09-03T20_05_34.006649-8pq9i6jt/ExampleValidator/output/4/, split: , id: 7)]
        ))

In [12]:
validation_dir = validate_stats.outputs['output'].get()[0].uri
anomalies_path = os.path.join(validation_dir, 'anomalies.pbtxt')

anomalies = tfdv.load_anomalies_text(anomalies_path)
tfdv.display_anomalies(anomalies)

In [13]:
_module_file = os.path.join(_root, 'utils.py')

transform = Transform(
    input_data=example_gen.outputs['examples'],
    schema=infer_schema.outputs['output'],
    module_file=_module_file)

context.run(transform)

W0903 20:05:44.390648 139893079226176 deprecation_wrapper.py:119] From /opt/interactive_pipeline/venv/lib/python3.6/site-packages/tfx/components/transform/executor.py:241: The name tf.logging.debug is deprecated. Please use tf.compat.v1.logging.debug instead.

W0903 20:05:44.394569 139893079226176 deprecation.py:323] From /opt/interactive_pipeline/venv/lib/python3.6/site-packages/tfx/components/transform/executor.py:406: Schema (from tensorflow_transform.tf_metadata.dataset_schema) is deprecated and will be removed in a future version.
Instructions for updating:
Schema is a deprecated, use schema_utils.schema_from_feature_spec to create a `Schema`
W0903 20:05:44.546715 139893079226176 deprecation.py:323] From /opt/interactive_pipeline/venv/lib/python3.6/site-packages/tensorflow/python/saved_model/signature_def_utils_impl.py:201: build_tensor_info (from tensorflow.python.saved_model.utils_impl) is deprecated and will be removed in a future version.
Instructions for updating:
This functi

ExecutionResult(
    component_id: Transform
    execution_id: 5
    outputs:
        transform_output: Channel(
            type_name: TransformPath
            artifacts: [Artifact(type_name: TransformPath, uri: /tmp/tfx-interactive-2019-09-03T20_05_34.006649-8pq9i6jt/Transform/transform_output/5/, split: , id: 8)]
        )
        transformed_examples: Channel(
            type_name: ExamplesPath
            artifacts: [Artifact(type_name: ExamplesPath, uri: /tmp/tfx-interactive-2019-09-03T20_05_34.006649-8pq9i6jt/Transform/transformed_examples/5/train/, split: train, id: 9)
            Artifact(type_name: ExamplesPath, uri: /tmp/tfx-interactive-2019-09-03T20_05_34.006649-8pq9i6jt/Transform/transformed_examples/5/eval/, split: eval, id: 10)]
        ))

In [14]:
trainer = Trainer(
    module_file=_module_file,
    transformed_examples=transform.outputs['transformed_examples'],
    schema=infer_schema.outputs['output'],
    transform_output=transform.outputs['transform_output'],
    train_args=trainer_pb2.TrainArgs(num_steps=1000),
    eval_args=trainer_pb2.EvalArgs(num_steps=500))

context.run(trainer)

W0903 20:05:45.988995 139893079226176 estimator.py:1811] Using temporary folder as model directory: /tmp/tmpufl9did_
W0903 20:05:45.996151 139893079226176 deprecation.py:323] From /opt/interactive_pipeline/venv/lib/python3.6/site-packages/tensorflow/python/training/training_util.py:236: Variable.initialized_value (from tensorflow.python.ops.variables) is deprecated and will be removed in a future version.
Instructions for updating:
Use Variable.read_value. Variables in 2.X are initialized automatically both in eager and graph (inside tf.defun) contexts.
W0903 20:05:46.009501 139893079226176 deprecation.py:323] From /opt/interactive_pipeline/venv/lib/python3.6/site-packages/tensorflow/python/data/experimental/ops/readers.py:835: parallel_interleave (from tensorflow.python.data.experimental.ops.interleave_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use `tf.data.Dataset.interleave(map_func, cycle_length, block_length, num_parallel_calls=tf.data.e

ExecutionResult(
    component_id: Trainer
    execution_id: 6
    outputs:
        output: Channel(
            type_name: ModelExportPath
            artifacts: [Artifact(type_name: ModelExportPath, uri: /tmp/tfx-interactive-2019-09-03T20_05_34.006649-8pq9i6jt/Trainer/output/6/, split: , id: 11)]
        ))

In [15]:
OVERALL_SLICE_SPEC = evaluator_pb2.SingleSlicingSpec()

ALL_SPECS = [
    OVERALL_SLICE_SPEC,
]

In [16]:
model_analyzer = Evaluator(
    examples=example_gen.outputs['examples'],
    model_exports=trainer.outputs['output'],
    feature_slicing_spec=evaluator_pb2.FeatureSlicingSpec(
        specs=ALL_SPECS
    ))

context.run(model_analyzer)

W0903 20:05:53.158023 139893079226176 deprecation.py:323] From /opt/interactive_pipeline/venv/lib/python3.6/site-packages/tensorflow_model_analysis/eval_saved_model/load.py:163: load (from tensorflow.python.saved_model.loader_impl) is deprecated and will be removed in a future version.
Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.loader.load or tf.compat.v1.saved_model.load. There will be a new function for importing SavedModels in Tensorflow 2.0.
W0903 20:05:53.279790 139893079226176 deprecation.py:323] From /opt/interactive_pipeline/venv/lib/python3.6/site-packages/tensorflow_model_analysis/eval_saved_model/graph_ref.py:189: get_tensor_from_tensor_info (from tensorflow.python.saved_model.utils_impl) is deprecated and will be removed in a future version.
Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.utils.get_tensor_from

ExecutionResult(
    component_id: Evaluator
    execution_id: 7
    outputs:
        output: Channel(
            type_name: ModelEvalPath
            artifacts: [Artifact(type_name: ModelEvalPath, uri: /tmp/tfx-interactive-2019-09-03T20_05_34.006649-8pq9i6jt/Evaluator/output/7/, split: , id: 12)]
        ))

In [17]:
PATH_TO_RESULT = model_analyzer.outputs['output'].get()[0].uri
tfma_result = tfma.load_eval_result(PATH_TO_RESULT)
tfma.view.render_slicing_metrics(tfma_result)

SlicingMetricsViewer(config={'weightedExamplesColumn': 'post_export_metrics/example_count'}, data=[{'slice': '…

In [18]:
model_validator = ModelValidator(
    examples=example_gen.outputs['examples'],
    model=trainer.outputs['output'])

context.run(model_validator)

ExecutionResult(
    component_id: ModelValidator
    execution_id: 8
    outputs:
        blessing: Channel(
            type_name: ModelBlessingPath
            artifacts: [Artifact(type_name: ModelBlessingPath, uri: /tmp/tfx-interactive-2019-09-03T20_05_34.006649-8pq9i6jt/ModelValidator/blessing/8/, split: , id: 13)]
        ))

In [ ]:
pusher = Pusher(
    model_export=trainer.outputs['output'],
    model_blessing=model_validator.outputs['blessing'],
    push_destination=pusher_pb2.PushDestination(
        filesystem=pusher_pb2.PushDestination.Filesystem(
            base_directory=_serving_model_dir)))

context.run(pusher)